In [ ]:
# Imports
import os
from typing import cast
from dotenv import load_dotenv
from agents import Agent, Runner, OpenAIChatCompletionsModel, AsyncOpenAI

# Arize Phoenix Tracer
from phoenix.otel import register
from opentelemetry import trace as otel_trace


# Markdown output display
from IPython.display import Markdown, display

In [69]:
# Cast to String
def env_to_str(env: str) -> str:
    return cast(str, os.getenv(env))

In [70]:
# Load Environment Variables
load_dotenv(override=True)

# Ollama Environment Variables
ollama_api_key = env_to_str('OLLAMA_API_KEY')
ollama_base_url = env_to_str('OLLAMA_BASE_URL')
model_gemma3 = env_to_str('MODEL_GEMMA3')

# Phoenix Collector Variables
phoenix_collector_endpoint = env_to_str("PHOENIX_COLLECTOR_ENDPOINT") or "http://localhost:6006/v1/traces"

In [71]:
# Configure Phoenix Tracer
try:
    tracer_provider = register(
    project_name="ollama_phoenixv2",
    auto_instrument=True,
    endpoint=phoenix_collector_endpoint,
    set_global_tracer_provider=True
)
    display(Markdown("✅ Phoenix tracer registered successfully"))
except Exception as e:
    display(Markdown(f"❌ Phoenix registration failed: {e}"))

Overriding of current TracerProvider is not allowed
DependencyConflict: requested: "openai-agents >= 0.1.0" but found: "openai-agents 0.0.17"
DependencyConflict: requested: "google-genai" but found: "None"


🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: ollama_phoenixv2
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: http://localhost:6006/v1/traces
|  Transport: HTTP + protobuf
|  Transport Headers: {}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  ⚠️ WARNING: It is strongly advised to use a BatchSpanProcessor in production environments.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



✅ Phoenix tracer registered successfully

In [72]:
# Ollama Model
ollama_model = OpenAIChatCompletionsModel(
    model=model_gemma3,
    openai_client=AsyncOpenAI(
        api_key=ollama_api_key, 
        base_url=ollama_base_url
    )
)

# Ollama Agent
ollama_agent = Agent(
    name="The Jokester", 
    instructions="You are a joke teller.", 
    model=ollama_model
)

In [73]:
# Test function with explicit tracing
async def ollama_tracer():
    tracer = otel_trace.get_tracer(__name__)
    with tracer.start_as_current_span("Chicken Joke"):
        
        # Add custom attributes to the trace
        current_span = otel_trace.get_current_span()
        current_span.set_attribute("user.request", "Tell a joke about chickens and roads")
        current_span.set_attribute("agent.name", "The Jokester")
        current_span.set_attribute("model.name", model_gemma3 or "unknown")
        
        result = await Runner.run(ollama_agent, "Tell a joke about chickens and roads.")
        
        # Add response attributes
        current_span.set_attribute("response.length", len(result.final_output))
        current_span.set_attribute("response.preview", result.final_output[:100])
        
        display(Markdown(result.final_output))
        return result.final_output
 

In [74]:
result = await ollama_tracer()

Why did the chicken cross the playground? 

To get to the other slide! 



😂🐔

**IMPORTANT:**
- **THIS EXAMPLE -DOES- USE TRACE**
- **ARIZE PHOENIX TRACE:**
  - **OLLAMA**
  - **?GEMINI**